In [3]:
from sed1.processes import sed_process_registry
from sed1.core import (
    serialize_instance, generate_script, generate_composite_process, get_process_schema)
from sed1 import ports, annotate, register
from bigraph_viz import plot_bigraph, plot_flow, pf
import numpy as np

In [4]:
# add sed processes to global namespace
sed_process_registry.activate_all(globals())
print(pf(sed_process_registry.registry))

{ 'concentration_control_coefficient': { 'address': <function concentration_control_coefficient at 0x29bb1b700>,
                                         'annotation': 'sed:is_steady_state',
                                         'ports': { 'inputs': { 'model': 'Model',
                                                                'parameter': 'string',
                                                                'variable': 'string'},
                                                    'outputs': { 'coefficient': 'float'}}},
  'data_description': { 'address': <function data_description at 0x29bb1b430>,
                        'annotation': 'sed:data_description',
                        'ports': { 'inputs': { 'data_file': 'str',
                                               'file_format': 'str'},
                                   'outputs': {'data': 'pd.DataFrame'}}},
  'is_steady_state': { 'address': <function is_steady_state at 0x29bb1b670>,
                       'annotati

In [5]:
# SED requires some more primitive operations for this

@ports({
    'inputs': {
        'trials': 'int',
    },
    'outputs': {
        'out': 'Any'}})
@annotate('sed:range_iterator')
def range_iterator(trials, func):
    result = None
    for i in range(r):
        result = func(result)
    return result


@ports({
    'outputs': {
        'value': 'float'}})
@annotate('sed:loop')
def random_number():
    return random.random()


@ports({
    'inputs': {'a': 'int', 'b': 'int'}, 
    'outputs': {'c': 'int'}})
@annotate('math:add')
def add(a, b):
    return a + b


@ports({'inputs': {'condition': 'bool'}})
@annotate('sed:condition')
def report_error(condition):
    if condition:
        raise Exception('Error!')

        
# add to the registry
functions = [
    range_iterator,
    random_number,
    add,
    report_error,
]

sed_process_registry = register(functions, sed_process_registry)
# print(pf(sed_process_registry.registry))

```
# This experiment randomizes two parameters, and
# computes the control coefficient for
# every reaction step in the model
# Outputs a 2D array, first and second column are k1 and k2
# remaining columns are the control coefficients
result1 is 1D array
result2 = is 1D array
trials = 10
load model (xyz.xml)  # This experiment only applies to SBML models
configure simulator (....)
for i = 0 to trials - 1 do
     model.k1 = random()
     model.k2 = random()
     for j = 0 to numReactions - 1 do
         compute Steady state
         if steadystate found:
            result1[i][0] = model.k1
            result1[i][1] = model.k2
            result1[i][j+2] = compute control coefficient (J1, j)
            sum = sum + result1[i][i+2]
          else
            handle error condition in some way,
            eg ignore and continue and record bad result in results array
        result2[i] = sum  # to check that results are ok
output table result1
output table result2
```

In [6]:
import numpy as np
import random

trials = 10
num_reactions = 10
result1 = np.empty((trials, numReactions + 2))
result2 = np.empty(trials)
model_path = "susceptible_zombie.xml"
model = sbml_model_from_path(path_to_sbml=model_path)  # Load the model
susceptible_id = 'S'
zombie_id = 'Z'
rate_param_id = 'k1'

for i in range(trials):
    k1 = random_number()
    k2 = random_number()
    model_set_value(model, susceptible_id, k1)
    model_set_value(model, zombie_id, k2)
    tolerance = 0.01
    total = 0
    for j in range(num_reactions):
        ss = steady_state_values(model, selection_list=[susceptible_id, zombie_id])
        steady_state_found = is_steady_state(model, tolerance)
        if steady_state_found:  
            s = model_get_value(model, susceptible_id)
            z = model_get_value(model, zombie_id)
            result1[i][0] = s
            result1[i][1] = z
            result1[i][j+2] = concentration_control_coefficient(model, susceptible_id, rate_param_id)
            total += result1[i][j+2]
        else:
            report_error(True)
    result2[i] = total

print("Result1:")
print(result1)
print("Result2:")
print(result2)

NameError: name 'numReactions' is not defined

In [ ]:
trials = 10
num_reactions = 10

# Example function calls with input and output arguments
instance1 = {
    'trials': trials,
    'num_reactions': num_reactions, 
    'result1': np.empty((trials, num_reactions + 2)), 
    'result2': np.empty(trials), 
    'susceptible_id': '"S"',
    'zombie_id': '"Z"',
    'rate_param_id': '"k1"',
    'model_path': '"susceptible_zombie.xml"',
    'trials': '"susceptible_zombie.csv"',
    'sbml_model_from_path': {
        '_type': 'sed:sbml_model_from_path',
        'wires': {
            'path_to_sbml': 'model_path',
            'model': 'model_instance'
        },
    },
    'range_iterator': {
        '_type': 'sed:range_iterator',
        'wires': {
            'trials': 'trials',
        },
        'range_iterator': {
            '_type': 'sed:range_iterator',
            'wires': {
                'trials': 'trials',
            },

        }
    }
}

# fill(instance)  # autocomplete
# validate_schema(instance) # TODO -- demonstrate validation

# Serialize to JSON
json_str1 = serialize_instance(instance1)
print(json_str1)

In [ ]:
# Generate the script
script = generate_script(json_str1, sed_process_registry)
print(script)

In [ ]:
# execute the script
exec(script)